In [1]:
import pandas as pd
import numpy as np
import sys
import hdbscan

In [23]:
# Load up code to run ICIM
# Available from : https://github.com/felixhorns/FlyPN
sys.path.append("../code/ICIM")
import sct
reload(sct)
# Used because of nature of ICIM library
pd.options.mode.chained_assignment = None  # default='warn'

In [16]:
dc_genes = pd.read_csv("../data/07_dropclust_data/dropclust_pca_genes.csv", header=None)
dc_genes.columns = ['symbol']
# Remove Hsp's

In [22]:
dc_genes = dc_genes[~dc_genes.symbol.str.contains('Hsp')]

In [30]:
dc_genes = list(dc_genes.symbol)

In [24]:
# Set up to pull out the list of genes in the pruned expression table
df = pd.read_csv("../data/02_filtered_kenyon_cells/CLEAN_LOG2TRANSFORM_kenyon_cells.csv", index_col='symbol')

In [25]:
df.shape

(17473, 2848)

In [26]:
# ended up with a random row nan whose entire row is 0 counts - FAKE GENE
print('Shape of df with nan index : ' + str(df[df.index.isna()].shape))
print('total sum of expression values : ' + str(sum(df[df.index.isna()].iloc[0])))
print('drop row')
df = df[~df.index.isna()]

Shape of df with nan index : (1, 2848)
total sum of expression values : 0.0
drop row


In [27]:
df.shape

(17472, 2848)

In [31]:
# Generate a filtered df based on these genes
f_df = df.copy(deep=True)
f_df = f_df.loc[dc_genes]

In [33]:
f_df.head()

,ACATACGAGGGCTTCC-DGRP-551_0d_r1,ACCCACTTCACTCTTA-DGRP-551_0d_r1,ACCGTAAAGATAGTCA-DGRP-551_0d_r1,ACTTACTAGTGGTAAT-DGRP-551_0d_r1,ACTTGTTCATGGTTGT-DGRP-551_0d_r1,ACTTTCATCAATAAGG-DGRP-551_0d_r1,AGATCTGCAACAACCT-DGRP-551_0d_r1,AGATCTGTCTACTCAT-DGRP-551_0d_r1,AGCGGTCCATTTCACT-DGRP-551_0d_r1,AGCTCTCGTTTGACTG-DGRP-551_0d_r1,...,GGCAATTCATGGATGG-w1118_15d_r1,GTCCTCAGTTGCGCAC-w1118_15d_r1,GTGCAGCGTACCGTAT-w1118_15d_r1,TATGCCCTCTATGTGG-w1118_15d_r1,TCATTACAGAGGTTGC-w1118_15d_r1,TCGAGGCAGCTATGCT-w1118_15d_r1,TTCTACATCAGTGTTG-w1118_15d_r1,AAGTCTGTCTGGTTCC-w1118_30d_r1,ACATACGTCGGATGGA-w1118_30d_r1,GAAATGACAAGACACG-w1118_30d_r1
symbol,,,,,,,,,,,,,,,,,,,,,
mamo,2.000000,4.392317,0.0,3.906891,2.321928,4.087463,4.523562,4.857981,1.000000,3.321928,...,1.584963,3.700440,0.000000,2.321928,2.807355,2.321928,3.906891,4.700440,2.321928,5.247928
pros,1.584963,3.906891,5.0,3.584963,1.584963,2.807355,3.700440,1.584963,0.000000,2.584963,...,1.000000,2.807355,4.392317,2.584963,3.321928,2.321928,2.000000,2.807355,2.000000,3.169925
CG32264,1.000000,1.000000,2.0,1.000000,0.000000,1.584963,2.000000,2.584963,2.000000,1.000000,...,1.584963,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,2.321928,1.000000,2.000000
alphagamma-element:CR32865,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.584963,0.000000,0.000000,0.000000
Rbp6,3.321928,3.700440,0.0,2.807355,3.459432,2.321928,2.807355,4.169925,2.807355,2.000000,...,2.000000,0.000000,0.000000,3.459432,0.000000,1.584963,2.807355,2.321928,1.000000,2.584963


In [36]:
f_df.shape

(193, 2848)

In [34]:
# Check to make sure each gene is present in at least 3 cells
def check_min_num_cells(row):
    num_pos = 0
    for gene_count in row:
        if gene_count > 0:
            num_pos += 1
            if num_pos >= 3:
                return True
    return False
   

min_num_cells = f_df.apply(check_min_num_cells, axis=1)
f_df = f_df[min_num_cells]

In [35]:
f_df.shape

(193, 2848)

In [45]:
# load up metadata that corresponds to full df
metadata = pd.read_csv("../data/02_filtered_kenyon_cells/metadata_kenyon_cells.csv")
metadata = metadata.set_index("CellID")

In [46]:
# Calculate TSNE
reload(sct)
from sklearn.manifold import TSNE
myTSNE = sct.TSNE(f_df, df, metadata)
myTSNE.calc_TSNE(perplexity=10, learning_rate=250, early_exaggeration=4.0, method="exact", random_state=1)

[t-SNE] Computed conditional probabilities for sample 1000 / 2848
[t-SNE] Computed conditional probabilities for sample 2000 / 2848
[t-SNE] Computed conditional probabilities for sample 2848 / 2848
[t-SNE] Mean sigma: 0.090996
[t-SNE] Iteration 50: error = 21.8934857, gradient norm = 0.0296051 (50 iterations in 15.246s)
[t-SNE] Iteration 100: error = 20.9777141, gradient norm = 0.0071826 (50 iterations in 14.751s)
[t-SNE] Iteration 150: error = 20.7919601, gradient norm = 0.0051248 (50 iterations in 14.786s)
[t-SNE] Iteration 200: error = 20.7025968, gradient norm = 0.0060609 (50 iterations in 19.596s)
[t-SNE] Iteration 250: error = 20.6529200, gradient norm = 0.0094798 (50 iterations in 15.025s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 20.652920
[t-SNE] Iteration 300: error = 2.6697218, gradient norm = 0.0012354 (50 iterations in 14.999s)
[t-SNE] Iteration 350: error = 2.4121419, gradient norm = 0.0005695 (50 iterations in 14.979s)
[t-SNE] Iteration 400: err

In [47]:
# Using HDBSCAN to call clusters automatically, generate metadata for them
clusterer = hdbscan.HDBSCAN(min_cluster_size=30, min_samples=3).fit(myTSNE.X_tsne)
labels_HDBSCAN = clusterer.labels_

In [48]:
print "Clusters", max(labels_HDBSCAN)+1

Clusters 4


In [49]:
zero_axis = myTSNE.X_tsne[:,0]
one_axis = myTSNE.X_tsne[:,1]
myTSNE.df_libs['x'] = zero_axis
myTSNE.df_libs['y'] = one_axis
myTSNE.df_libs['hdb_clust'] = labels_HDBSCAN

In [50]:
def gen_kc_type(row):
    if int(row.cell_type_id) == 8:
        return 'G-KC'
    elif int(row.cell_type_id) == 22:
        return 'a/b-KC'
    elif int(row.cell_type_id) == 28:
        return "a'/b'-KC"
    
myTSNE.df_libs['subtype'] = myTSNE.df_libs.apply(gen_kc_type, axis=1)
myTSNE.df_libs.to_csv("../data/07_dropclust_data/dc_gene_TSNE.csv")

In [51]:
myTSNE.df_libs

,Age,Gender,Genotype,Replicate,nGene,nUMI,cell_type_id,is_kc,x,y,hdb_clust,subtype
CellID,,,,,,,,,,,,
ACATACGAGGGCTTCC-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1328,3340.0,8.0,1,60.906742,-43.021698,1,G-KC
ACCCACTTCACTCTTA-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1613,4580.0,8.0,1,43.453411,-28.135614,1,G-KC
ACCGTAAAGATAGTCA-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1466,4349.0,22.0,1,-61.779900,54.877357,2,a/b-KC
ACTTACTAGTGGTAAT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1174,2942.0,8.0,1,41.978565,-37.542160,1,G-KC
ACTTGTTCATGGTTGT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1410,3620.0,8.0,1,72.495132,-31.920502,1,G-KC
ACTTTCATCAATAAGG-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1378,3995.0,8.0,1,53.749554,-43.828163,1,G-KC
AGATCTGCAACAACCT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1578,4379.0,8.0,1,46.485527,-40.284489,1,G-KC
AGATCTGTCTACTCAT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1592,4639.0,28.0,1,-30.070433,-63.706177,0,a'/b'-KC
AGCGGTCCATTTCACT-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1165,2920.0,28.0,1,17.261841,41.965294,3,a'/b'-KC
